In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import h5py
from scipy import io
from scipy import signal
import os
import glob
import csv
import pywt
from biosppy.signals import ecg

In [2]:
arr = []
with open('training2017/REFERENCE.csv', newline='') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
     for row in spamreader:
         spam = row[0].split(',')
         if spam[1] == 'A':
            arr.append(spam[0])
print (len(arr))            

738


In [3]:
l = 738 
d = 9000
r = 256
data = []
length = []
oversampling = 2
for i in range(l):
    
    db = io.loadmat('training2017/' + arr[i]) 
    for o in range(oversampling):
        dummy = db['val'][0, :]
        length.append(len(dummy))
        if len(dummy) < 9000:
            dummy = np.append(dummy, dummy)
            dummy = np.append(dummy, dummy)
        dummy = dummy[0:d]    
        min = np.amin(dummy)
        max = np.amax(dummy)    
        ddata = np.copy((dummy-min)/(max-min))
        ddata = signal.resample(ddata, r)
        data.append(ddata)
        dummy = -dummy
        min = np.amin(dummy)
        max = np.amax(dummy)    
        ddata = np.copy((dummy-min)/(max-min))
        ddata = signal.resample(ddata, r)
        data.append(ddata)    
data = np.asarray(data).reshape(len(data), r, 1)
seq_in_af = np.copy(data)
print (seq_in_af.shape)  
#print (length)

(2952, 256, 1)


In [4]:
arr = []
with open('training2017/REFERENCE.csv', newline='') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
     for row in spamreader:
         spam = row[0].split(',')
         if spam[1] == 'N':
            arr.append(spam[0])
print (len(arr))            

5050


In [5]:
l = 5050 
d = 9000
r = 256
data = []
length = []
for i in range(l):
    
    db = io.loadmat('training2017/' + arr[i]) 
    dummy = db['val'][0, :]
    length.append(len(dummy))
    if len(dummy) < 9000:
        dummy = np.append(dummy, dummy)
        dummy = np.append(dummy, dummy)
    dummy = dummy[0:d]    
    min = np.amin(dummy)
    max = np.amax(dummy)    
    ddata = np.copy((dummy-min)/(max-min))
    ddata = signal.resample(ddata, r)
    data.append(ddata)
    dummy = -dummy
    min = np.amin(dummy)
    max = np.amax(dummy)    
    ddata = np.copy((dummy-min)/(max-min))
    ddata = signal.resample(ddata, r)
    data.append(ddata)  
data = np.asarray(data).reshape(len(data), r, 1)
seq_in_n = np.copy(data)
print (seq_in_n.shape)  
#print (length)

(10100, 256, 1)


In [6]:
X = np.append(np.asarray(seq_in_n), np.asarray(seq_in_af), axis=0)
Y = np.zeros((len(seq_in_n)+len(seq_in_af)))
Y[len(seq_in_n):] = 1

idx = np.random.randint(0, len(X), size=len(X))
X = X[idx]
Y = Y[idx]

In [7]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional, ConvLSTM2D, MaxPooling1D, Conv1D, Flatten, UpSampling1D, ZeroPadding1D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, RepeatVector
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils
from keras.utils import plot_model
from keras.regularizers import l1_l2
import keras.backend as K
from keras.models import model_from_json
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [8]:
def classifier():
    
    model = Sequential()           
    model.add(LSTM(128, input_shape=(r, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(0.0002, 0.5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [10]:
n_split = 5
for train_index, test_index in KFold(n_split).split(X):
    
  x_train, x_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  model = classifier()
  model.fit(x_train, y_train, epochs=2)
  print('Model evaluation ', model.evaluate(x_test, y_test))

Epoch 1/2
10441/10441 [==============================] - 92s 9ms/step - loss: 0.5485 - acc: 0.7747
Epoch 2/2
2611/2611 [==============================] - 4s 2ms/step
Model evaluation  [0.5427310835443114, 0.7694369971364086]
Epoch 1/2
10441/10441 [==============================] - 82s 8ms/step - loss: 0.5480 - acc: 0.7766
Epoch 2/2
2611/2611 [==============================] - 4s 2ms/step
Model evaluation  [0.5450942354246101, 0.7656070469257613]
Epoch 1/2
10442/10442 [==============================] - 82s 8ms/step - loss: 0.5456 - acc: 0.7765
Epoch 2/2
2610/2610 [==============================] - 4s 1ms/step
Model evaluation  [0.5633804176959042, 0.7593869732714248]
Epoch 1/2
10442/10442 [==============================] - 68s 7ms/step - loss: 0.5483 - acc: 0.7720
Epoch 2/2
2610/2610 [==============================] - 4s 2ms/step
Model evaluation  [0.5225635532446748, 0.7839080458399893]
Epoch 1/2
10442/10442 [==============================] - 68s 7ms/step - loss: 0.5555 - acc: 0.7660
E

solution:

https://www.mathworks.com/help/signal/examples/classify-ecg-signals-using-long-short-term-memory-networks.html